In [1]:
#hide
#default_exp showdoc
#default_cls_lvl 3
from nbdev.showdoc import show_doc

In [2]:
#export
from nbdev.imports import *
from nbconvert import HTMLExporter
from fastcore.utils import IN_NOTEBOOK

if IN_NOTEBOOK:
    from IPython.display import Markdown,display
    from IPython.core import page

In [3]:
#export
import sys
import os

SCRIPT_DIR = os.path.dirname(os.getcwd()) + '/chisel_nbdev'
sys.path.append(os.path.normpath(SCRIPT_DIR))
from export_scala import *
from export_scala import get_nbdev_module
from sync_scala import *

# Show doc

> Functions to show the doc cells in notebooks

All the automatic documentation of functions and classes are generated with the `show_doc` function. It displays the name, arguments, docstring along with a link to the source code on GitHub.

## Gather the information

The inspect module lets us know quickly if an object is a function or a class but it doesn't distinguish classes and enums.

In [4]:
#export
def is_enum(cls):
    "Check if `cls` is an enum or another type of class"
    return type(cls) in (enum.Enum, enum.EnumMeta)

In [5]:
e = enum.Enum('e', 'a b')
assert is_enum(e)
assert not is_enum(e.__class__)
assert not is_enum(int)

### Links to documentation

In [6]:
#export
re_digits_first = re.compile('^[0-9]+[a-z]*_')

looks to nbdev/_nbdeb.py where our index is stored.

In [7]:
#export
def is_doc_name(name):
    "Test if `name` corresponds to a notebook that could be converted to a doc page"
    for f in Config().path("nbs_path").glob(f'*{name}.ipynb'):
        if re_digits_first.sub('', f.name) == f'{name}.ipynb': return True
    return False

In [8]:
test_eq(is_doc_name('flaags'),False)
test_eq(is_doc_name('export_scala'),True)
test_eq(is_doc_name('sync_scala'),True)
test_eq(is_doc_name('test_scala'),True)
test_eq(is_doc_name('test'),True)
test_eq(is_doc_name('ToImport'),True)
test_eq(is_doc_name('test.Operator'),False) # only notebook names

In [9]:
#export
def doc_link(name, include_bt=True):
    "Create link to documentation for `name`."
    cname = f'`{name}`' if include_bt else name
    try:
        #Link to modules
        if is_doc_name(name): return f"[{cname}]({Config().doc_baseurl}{name}.html)"
        #Link to local functions
        try_local = source_nb(name, is_name=True)
        if try_local:
            page = re_digits_first.sub('', try_local).replace('.ipynb', '')
            return f'[{cname}]({Config().doc_baseurl}{page}.html#{name})'
        ##Custom links
        mod = get_nbdev_module()
        link = mod.custom_doc_links(name)
        return f'[{cname}]({link})' if link is not None else cname
    except Exception as e: print(e); return cname

This function will generate links for modules (pointing to the html conversion of the corresponding notebook) and functions (pointing to the html conversion of the notebook where they were defined, with the first anchor found before). If the function/module is not part of the library you are writing, it will call the function `custom_doc_links` generated in `_nbdev` (you can customize it to your needs) and just return the name between backticks if that function returns `None`.

For instance, fastai has the following `custom_doc_links` that tries to find a doc link for `name` in fastcore then nbdev (in this order):
``` python
def custom_doc_links(name): 
    from nbdev.showdoc import try_external_doc_link
    return try_external_doc_link(name, ['fastcore', 'nbdev'])
```

Please note that module links only work if your notebook names "correspond" to your module names:

| Notebook name      | Doc name       | Module name | Module file  | Can doc link? |
|--------------------|----------------|-------------|--------------|---------------|
| export.ipynb       | export.html    | export      | export.py    | Yes           |
| 00_export.ipynb    | export.html    | export      | export.py    | Yes           |
| 00a_export.ipynb   | export.html    | export      | export.py    | Yes           |
| export_1.ipynb     | export_1.html  | export      | export.py    | No            |
| 03_data.core.ipynb | data.core.html | data.core   | data/core.py | Yes           |
| 03_data_core.ipynb | data_core.html | data.core   | data/core.py | No            |

In [10]:
test_eq(doc_link('export_scala'), f'[`export_scala`](/export_scala.html)')
test_eq(doc_link('test_scala'), f'[`test_scala`](/test_scala.html)')
test_eq(doc_link('sync_scala'), f'[`sync_scala`](/sync_scala.html)')
test_eq(doc_link('test'), f'[`test`](/test.html)')
test_eq(doc_link('Operator'), f'[`Operator`](/test.html#Operator)')
test_eq(doc_link('FuncToNewScript'), f'[`FuncToNewScript`](/test.html#FuncToNewScript)')

In [11]:
#export
_re_backticks = re.compile(r"""
# Catches any link of the form \[`obj`\](old_link) or just `obj`,
#   to either update old links or add the link to the docs of obj
\[`      #     Opening [ and `
([^`]*)  #     Catching group with anything but a `
`\]      #     ` then closing ]
(?:      #     Beginning of non-catching group
\(       #       Opening (
[^)]*    #       Anything but a closing )
\)       #       Closing )
)        #     End of non-catching group
|        # OR
`        #     Opening `
([^`]*)  #       Anything but a `
`        #     Closing `
""", re.VERBOSE)

In [12]:
#export
def add_doc_links(text, elt=None):
    "Search for doc links for any item between backticks in `text` and insert them"
    def _replace_link(m): 
        try: 
            if m.group(2) in inspect.signature(elt).parameters: return f'`{m.group(2)}`'
        except: pass
        return doc_link(m.group(1) or m.group(2))
    return _re_backticks.sub(_replace_link, text)

This function not only add links to backtick keywords, it also update the links that are already in the text (in case they have changed).

In [13]:
tst = add_doc_links('This is an example of `Operator`')
test_eq(tst, "This is an example of [`Operator`](/test.html#Operator)")
tst = add_doc_links('This is an example of [`Operator`](old_link.html)')
test_eq(tst, "This is an example of [`Operator`](/test.html#Operator)")

As important as the source code, we want to quickly jump to where the function is defined when we are in a development notebook.

In [14]:
#export
_re_header = re.compile(r"""
# Catches any header in markdown with the title in group 1
^\s*  # Beginning of text followed by any number of whitespace
\#+   # One # or more
\s*   # Any number of whitespace
(.*)  # Catching group with anything
$     # End of text
""", re.VERBOSE)

In [15]:
#export
def colab_link(path):
    "Get a link to the notebook at `path` on Colab"
    cfg = Config()
    res = f'https://colab.research.google.com/github/{cfg.user}/{cfg.lib_name}/blob/{cfg.branch}/{cfg.path("nbs_path").name}/{path}.ipynb'
    display(Markdown(f'[Open `{path}` in Colab]({res})'))

In [16]:
colab_link('00_export_scala')

[Open `00_export_scala` in Colab](https://colab.research.google.com/github/ucsc-vama/chisel-nbdev/blob/master/nbs/00_export_scala.ipynb)

### Links to source

Configure a regex to just look for desired object/class/function/etc name

In [66]:
#export
def _make_class_func_by_name(find_name):
    return re.compile(fr'''\s*(abstract\s+class|case\s+class|class|object|trait|sealed\s+trait|implicit\s+def|def)\s+{find_name}\s*|\W+''', re.MULTILINE)

In [67]:
_operator_re = _make_class_func_by_name('Operator')
print(_operator_re)
print(re.search(_operator_re, " abstract class Operator(a : Int, b: Boolean = false)"))

re.compile('\\s*(abstract\\s+class|case\\s+class|class|object|trait|sealed\\s+trait|implicit\\s+def|def)\\s+Operator\\s*|\\W+', re.MULTILINE)
<re.Match object; span=(0, 24), match=' abstract class Operator'>


RegEx to extract out all of the params from a signature

In [159]:
#export
_re_get_params_list = re.compile(r'''
\((.*)\)
''', re.MULTILINE | re.VERBOSE)

In [160]:
test_eq(re.search(_re_get_params_list, " abstract class Operator(a : Int, b: Boolean = false)").group(), '(a : Int, b: Boolean = false)')
test_eq(re.search(_re_get_params_list, " class Foo( a: (Int, Int), b: (Int, Int), c: Boolean) = {}").group(), '( a: (Int, Int), b: (Int, Int), c: Boolean)')

Regex to extract out each param from a param list

In [215]:
#export
_re_get_scala_params = re.compile(r'''
\(.*:.*,|
,.*:.*,|
,.*:.*\)
''', re.MULTILINE | re.VERBOSE)

In [216]:
print(re.findall(_re_get_scala_params, '(a : Int, b: Boolean = false)'))

['(a : Int,']


In [106]:
#export
def get_nb_source_link(func, local=False):
    "Return a link to the notebook where `func` is defined."
#     func = _unwrapped_type_dispatch_func(func)
    pref = '' if local else Config().git_url.replace('github.com', 'nbviewer.jupyter.org/github')+ Config().path("nbs_path").name+'/'
    src = source_nb(func, is_name=True, return_all=True)
    if src is None: return '' 
    find_name,nb_name = src
    print(find_name)
    nb = read_nb(nb_name)
#     pat = re.compile(f'^{find_name}\s+=|^(def|class)\s+{find_name}\s*\(', re.MULTILINE)
    pat = _make_class_func_by_name(find_name)
    if len(find_name.split('.')) == 2:
        clas,pat = find_name.split('.') # split test.Add -> Add in situation where Add defined in > 1 modules
    for i,cell in enumerate(nb['cells']):
        if cell['cell_type'] == 'code':
            if re.search(pat, cell['source']):  break
    if re.search(pat, cell['source']) is None:
        return '' 
    header_pat = re.compile(r'^\s*#+\s*(.*)$')
    while i >= 0:
        cell = nb['cells'][i]
        if cell['cell_type'] == 'markdown' and _re_header.search(cell['source']):
            title = _re_header.search(cell['source']).groups()[0]
            anchor = '-'.join([s for s in title.split(' ') if len(s) > 0])
            return f'{pref}{nb_name}#{anchor}'
        i-=1
    return f'{pref}{nb_name}'

In [107]:
get_nb_source_link("Operator", local=True)
get_nb_source_link("test.Add", local=True)
get_nb_source_link("Add", local=True)
get_nb_source_link("TestObj", local=True)

Operator
test.Add
Add
TestObj


'test.ipynb#Test-Chisel-notebook'

You can pass a func name based as a string. `local` will return a local link, otherwise it will point to a the notebook on Google Colab.

In [108]:
#export
def nb_source_link(func, disp=True, local=True):
    "Show a relative link to the notebook where `func` is defined"
    link = get_nb_source_link(func, local=local)
    text = func if local else f'{func} (GitHub)'
    if disp: display(Markdown(f'[{text}]({link})'))
    else: return link

This function assumes you are in one notebook in the development folder, otherwise you can use `disp=False` to get the relative link. 

In [109]:
# test_eq(nb_source_link('check_re', disp=False), f'00_export.ipynb#Finding-patterns')
test_eq(nb_source_link("Operator", local=True, disp=False), f'test.ipynb#Test-Chisel-notebook')
test_eq(nb_source_link("Add", local=True, disp=False), f'ToImport.ipynb')
test_eq(nb_source_link("TestObj", local=True, disp=False), f'test.ipynb#Test-Chisel-notebook')

Operator
Add
TestObj


## Show documentation

In [110]:
#export
def show_doc(fname, doc_string=True, name=None, title_level=None, disp=True, default_cls_level=2):
    "Show documentation for element `elt`. Supported types: class, function, and enum."
    qname = fma,e
    if inspect.isclass(elt):
        if is_enum(elt): name,args = _format_enum_doc(elt, qname)
        else:            name,args = _format_cls_doc (elt, qname)
    elif callable(elt):  name,args = _format_func_doc(elt, qname)
    else:                name,args = f"<code>{qname}</code>", ''
    link = get_source_link(elt)
    source_link = f'<a href="{link}" class="source_link" style="float:right">[source]</a>'
    title_level = title_level or (default_cls_level if inspect.isclass(elt) else 4)
    doc =  f'<h{title_level} id="{qname}" class="doc_header">{name}{source_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n' if len(args) > 0 else '\n\n'
    if doc_string and inspect.getdoc(elt):
        s = inspect.getdoc(elt)
        # show_doc is used by doc so should not rely on Config
        try: monospace = (Config().get('monospace_docstrings') == 'True')
        except: monospace = False
        # doc links don't work inside markdown pre/code blocks
        s = f'```\n{s}\n```' if monospace else add_doc_links(s, elt)
        doc += s
    if disp: display(Markdown(doc))
    else: return doc